<a href="https://colab.research.google.com/github/musicjae/recommender-system/blob/main/Basic/RecSys_using_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Import

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

## Data Load

In [3]:
path = '/content/drive/MyDrive/Colab_Notebooks/dataset/recommendation/movies'

movie = pd.read_csv(path+'/ratings.csv',low_memory=False)
movie.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [4]:
movie = movie.sort_values(by='timestamp',ascending=True).reset_index(drop=True) # 시간순 정렬
movie.head(3)

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009


In [5]:
meta = pd.read_csv(path+'/movies_metadata.csv',low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [6]:
meta = meta.rename(columns={'id':'movieId'}) # movie dataframe과 병합하기 위해 칼럼명 맞춰줌
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie,meta[['movieId','original_title']], how='left',on='movieId') # Join
movie.head(2)

,userId,movieId,rating,timestamp,original_title
0,383,21,3.0,789652009,The Endless Summer
1,383,47,5.0,789652009,NaN


In [7]:
# 결측치 처리
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)
#각 유저가 어떤 영화를 보았는지 알아보기
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head(3)

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."


In [8]:
sen = []

for user_sen in agg['unique'].values:
    
    sen.append(list(map(str,user_sen))) # 전체 리스트 안에 담긴 원소가 string으로 변환되게 하기 위해 매핑
print(sen)

[['Jay and Silent Bob Strike Back', 'Vivement dimanche!', 'Rocky III', 'American Pie', 'My Tutor', 'Greed'], ['Terminator 3: Rise of the Machines', 'The Conversation', 'The Hours', '48 Hrs.', 'Back to the Future Part II', 'Silent Hill', 'Crustacés et coquillages', 'Lost in Translation', 'Night on Earth', "Dave Chappelle's Block Party", "Ocean's Eleven", 'Sissi', 'Live and Let Die', 'A Clockwork Orange', 'Солярис', 'Sommer vorm Balkon', 'La science des rêves', 'Trois couleurs : Rouge', 'Grbavica', 'Czlowiek z zelaza', 'Le Mépris', 'Batman Returns', 'Romeo + Juliet', 'Monsoon Wedding', 'Stand by Me', 'Lucky Number Slevin', 'Cat on a Hot Tin Roof', 'The Dark', 'The Devil Wears Prada', 'Lili Marleen', 'Star Trek IV: The Voyage Home', 'A Nightmare on Elm Street', 'Notting Hill', 'Once Were Warriors', 'Reservoir Dogs', '2001: A Space Odyssey', 'Rebecca', 'Psycho', 'The Poseidon Adventure', 'Batman Begins', 'Das weisse Rauschen', 'Big Fish', 'Wag the Dog', 'My Own Private Idaho', 'Monsters, I

## Word2vec

In [9]:
from gensim.models import Word2Vec

emb_model = Word2Vec(sen, size = 20, window = 5, min_count=1,workers=4,iter=100,sg=1)

In [10]:
emb_model.wv.most_similar(positive=['Spider-Man 2'],topn=10)

[('Snow Cake', 0.8479222059249878),
 ('Domicile Conjugal', 0.8104403018951416),
 ('Death Machine', 0.7401513457298279),
 ('Krull', 0.7304394245147705),
 ('薔薇の葬列', 0.726809561252594),
 ('Star Trek: Nemesis', 0.7231310606002808),
 ('Forrest Gump', 0.7210347652435303),
 ('Some Like It Hot', 0.7098243236541748),
 ('I, Robot', 0.7061198949813843),
 ('Fail-Safe', 0.7032958269119263)]